In [ ]:
import keras.backend as K
import tensorflow as tf
import numpy as np
import keras, sys, cv2
from keras.datasets import mnist
K.set_learning_phase(1)
img_rows, img_cols = 300, 400
num_classes = 10
from ipynb.fs.full.mnist_model import Model as MM
from keras.models import Model

In [6]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
print(x_test.shape)

(10000, 28, 28)


In [3]:
m = MM()


model=Model(inputs=[m.labels, m.inputs], outputs=[m.predictions, m.g, m.dd, m.gb_grad])
model.summary()
model.load_weights('my_model1.h5')

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_test = x_test / 255.
x_test = x_test.reshape((-1, 28, 28, 1))
y_test = keras.utils.to_categorical(y_test, num_classes)
print(y_test.shape)
for target_y_train_num in range(20):

    result = model.predict([
        y_test[target_y_train_num].reshape((1, 10)),
        x_test[target_y_train_num].reshape((1, 28, 28, 1))
    ])
    print('answer: ', K.eval(K.argmax(y_test[target_y_train_num])))
    print('prediction: ', K.eval(K.argmax(result[0])))

    conv_grad = result[1]
    conv_grad = conv_grad.reshape(conv_grad.shape[1:])
    conv_output = result[2]
    conv_output = conv_output.reshape(conv_output.shape[1:])
    input_grad = result[3]
    input_grad = input_grad.reshape(input_grad.shape[1:])
    gradRGB = gb_viz = input_grad

    from skimage.transform import resize
    import cv2

    # global average pooling
    weights = np.mean(conv_grad, axis = (0, 1))
    cam = np.zeros(conv_output.shape[0 : 2], dtype = np.float32)

    for i, w in enumerate(weights):
        cam += w * conv_output[:, :, i]

    cam = np.maximum(cam, 0)
    cam = cam / np.max(cam)
    cam = resize(cam, (28,28), preserve_range=True)

    img = x_test[target_y_train_num].astype(float)
    img -= np.min(img)
    img /= img.max()

    cam_heatmap = cv2.applyColorMap(np.uint8(255*cam), cv2.COLORMAP_JET)
    cam_heatmap = cv2.cvtColor(cam_heatmap, cv2.COLOR_BGR2RGB)


    cam = np.float32(cam.reshape((28, 28, 1))) * np.float32(img)
    cam = 255 * cam / np.max(cam)
    cam = np.uint8(cam)

    import matplotlib
    matplotlib.use('agg')
    import matplotlib.pyplot as plt

    plt.figure()
    img_int = (img * 255.).astype(int).reshape(img.shape[:2])
    plt.gray()
    plt.imshow(img_int)
    plt.savefig('mnist_cams/original_{}.png'.format(target_y_train_num))
    plt.close()

    plt.figure()
    plt.imshow(cam_heatmap)
    plt.savefig('mnist_cams/heatmap_{}.png'.format(target_y_train_num))
    plt.close()

    plt.figure()
    plt.imshow(img_int)
    plt.imshow(cam_heatmap, alpha=0.5)
    plt.savefig('mnist_cams/heatmap_overlaied_{}.png'.format(target_y_train_num))
    plt.close()

    gb_viz -= np.min(gb_viz)
    gb_viz /= gb_viz.max()
    img_int = (gb_viz * 255.).astype(int).reshape(img.shape[:2])
    imgplot = plt.imshow(img_int)
    plt.savefig('mnist_cams/grad-cam-backpropagation_{}.png'.format(target_y_train_num))
    plt.close()

    gd_gb = gb_viz * cam
    img_int = (gd_gb * 255.).astype(int).reshape(img.shape[:2])
    imgplot = plt.imshow(img_int)
    plt.savefig('mnist_cams/guided-grad-cam_{}.png'.format(target_y_train_num))
    plt.close()

ipynb.fs.full.mnist_model:41: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
/home/uttaran/anaconda3/lib/python3.6/site-packages/keras/legacy/layers.py:465: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)
ipynb.fs.full.mnist_model:42: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
ipynb.fs.full.mnist_model:43: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
imgs (InputLayer)               (None, 28, 28, 1)    0                                            
__________________________________________________________________________________________________
conv2d_3 (Conv2D)               (None, 26, 26, 8)    80          imgs[0][0]                       
__________________________________________________________________________________________________
max_pooling2d_4 (MaxPooling2D)  (None, 13, 13, 8)    0           conv2d_3[0][0]                   
__________________________________________________________________________________________________
conv2d_4 (Conv2D)               (None, 11, 11, 16)   1168        max_pooling2d_4[0][0]            
__________________________________________________________________________________________________
max_poolin

/home/uttaran/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


answer:  2
prediction:  [2]


/home/uttaran/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


answer:  1
prediction:  [1]
answer:  0
prediction:  [0]
answer:  4
prediction:  [4]
answer:  1
prediction:  [1]
answer:  4
prediction:  [4]
answer:  9
prediction:  [9]
answer:  5
prediction:  [5]
answer:  9
prediction:  [9]
answer:  0
prediction:  [0]
answer:  6
prediction:  [6]
answer:  9
prediction:  [9]
answer:  0
prediction:  [0]
answer:  1
prediction:  [1]
answer:  5
prediction:  [5]
answer:  9
prediction:  [9]
answer:  7
prediction:  [7]
answer:  3
prediction:  [3]
answer:  4
prediction:  [4]
